# Test neq code

In [1]:
from simtk import openmm, unit
from openmmtools.integrators import PeriodicNonequilibriumIntegrator, LangevinIntegrator, NonequilibriumLangevinIntegrator, AlchemicalNonequilibriumLangevinIntegrator
from perses.dispersed.utils import configure_platform
from openmmtools import utils, cache, states
import os
import pickle
from openmmtools.constants import kB
from perses.tests.utils import compute_potential_components
import numpy as np

conducting subsequent work with the following platform: CUDA


In [2]:
dir_num = 5
outdir = "/data/chodera/zhangi/perses_benchmark/neq/19/"
path = os.path.join(outdir, str(dir_num) + '/')
with open(os.path.join(path, f"{dir_num}_solvent.pickle"), "rb") as f:
    apo_htf = pickle.load(f)
    system = apo_htf.hybrid_system
    positions = apo_htf.hybrid_positions
    box_vectors = apo_htf.hybrid_system.getDefaultPeriodicBoxVectors()


In [3]:
# # Restrain system
# force = openmm.CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
# force.addGlobalParameter("k", 5.0*unit.kilocalories_per_mole/unit.angstroms**2)
# force.addPerParticleParameter("x0")
# force.addPerParticleParameter("y0")
# force.addPerParticleParameter("z0")
# for atom in apo_htf.hybrid_topology.atoms:
#     if atom.element.name != 'hydrogen':
#         force.addParticle(atom.index, positions[atom.index].value_in_unit(unit.nanometers))
# system.addForce(force)

8

In [3]:
system.getForce(4).getEnergyFunction()

'U_electrostatics; U_electrostatics = switch * c_RF * u_RF; switch = step(-t) + step(t) * step(1-t) * (1 + (t^3) * (-10 + t * (15 - (6 * t)))); t = (r_eff - r_switch) / (r_cutoff - r_switch); c_RF = 138.935456 * chargeProd; chargeProd = charge1 * charge2; charge1 = ((charge_old1 * ( 1 * environment_region1+lambda_0_electrostatics_old * alchemical_region_01 )) + (charge_new1 * ( 1 * environment_region1+lambda_0_electrostatics_new * alchemical_region_01 ))) * ( nonscale_lambda_electrostatics * nonscale_region1 ); charge2 = ((charge_old2 * ( 1 * environment_region2+lambda_0_electrostatics_old * alchemical_region_02 )) + (charge_new2 * ( 1 * environment_region2+lambda_0_electrostatics_new * alchemical_region_02 ))) * ( nonscale_lambda_electrostatics * nonscale_region2 ); u_RF = (1 / r_eff) + ((eps_RF - 1) * (r_eff^2)) / ((r_cutoff^3) * (1 + (2 * eps_RF))); r_eff = sqrt(r^2 + w^2); w = is_unique_old * r_cutoff * w_scale * (1 - lambda_0_electrostatics_old) + is_unique_new * w_scale * (1 - la

In [3]:
nsteps_eq = 25000
nsteps_neq = int(1*25000) # 100 ps
neq_splitting='V R H O R V'
timestep = 4.0 * unit.femtosecond
platform_name = 'CUDA'
temperature = 300.0 * unit.kelvin

kT = kB * temperature
beta = 1.0/kT

x = 'lambda'
ALCHEMICAL_FUNCTIONS = {'lambda_0_bonds_old': f'abs(1 - {x})',
                         'lambda_0_bonds_new': f'abs({x})',
                         'lambda_0_angles_old': f'abs(1 - {x})',
                         'lambda_0_angles_new': f'abs({x})',
                         'lambda_0_torsions_old': f'abs(1 - {x})',
                         'lambda_0_torsions_new': f'abs({x})',
                         'lambda_0_electrostatics_old':f'abs(1 - {x})',
                         'lambda_0_electrostatics_new': f'abs({x})',
                         'lambda_0_electrostatics_exceptions_old': f'abs(1 - {x})',
                         'lambda_0_electrostatics_exceptions_new': f'abs({x})',
                         'lambda_0_sterics_old': f'abs(1 - {x})',
                         'lambda_0_sterics_new': f'abs({x})',
                         'lambda_0_sterics_exceptions_old': f'abs(1 - {x})',
                         'lambda_0_sterics_exceptions_new': f'abs({x})'}

# Set up integrator
# integrator = PeriodicNonequilibriumIntegrator(ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)
integrator = LangevinIntegrator(temperature=temperature)


# Set up context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*box_vectors)
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)


In [4]:
# Minimize
openmm.LocalEnergyMinimizer.minimize(context)



In [7]:
# # Set all heavy atom masses to be 0
# heavy_atoms = []
# for atom in apo_htf.hybrid_topology.atoms:
#     if atom.element.name != 'hydrogen' and atom.residue.name not in ['HOH', 'Na+', 'Cl-']:
#         system.setParticleMass(atom.index, 0.0)
#         heavy_atoms.append(atom.index)

# for i in range(system.getNumConstraints() - 1, -1, -1):
#     p1, p2, distance = system.getConstraintParameters(i)
#     if p1 in heavy_atoms or p2 in heavy_atoms:
#         system.removeConstraint(i)


In [5]:
# # Restrain system
# force = openmm.CustomExternalForce("k*((x-x0)^2+(y-y0)^2+(z-z0)^2)")
# force.addGlobalParameter("k", 5.0*unit.kilocalories_per_mole/unit.angstroms**2)
# force.addPerParticleParameter("x0")
# force.addPerParticleParameter("y0")
# force.addPerParticleParameter("z0")
# for atom in apo_htf.hybrid_topology.atoms:
#     if atom.element.name != 'hydrogen':
#         force.addParticle(atom.index, positions[atom.index].value_in_unit(unit.nanometers))
# system.addForce(force)

8

In [5]:
# Set positions to be minimized positions
positions = context.getState(getPositions=True).getPositions()


In [11]:
heavy_atoms = []
for atom in apo_htf.hybrid_topology.atoms:
    if atom.element.name != 'hydrogen':
        heavy_atoms.append(atom.index)

In [13]:
# Add RMSD force
from simtk.openmm import RMSDForce
rmsd_force = RMSDForce(positions, heavy_atoms)
apo_htf.hybrid_system.addForce(rmsd_force)

8

In [14]:

# Set up new integrator
integrator = PeriodicNonequilibriumIntegrator(ALCHEMICAL_FUNCTIONS, nsteps_eq, nsteps_neq, neq_splitting, timestep=timestep, temperature=temperature)

# Set up new context
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')
context = openmm.Context(system, integrator, platform)
context.setPeriodicBoxVectors(*box_vectors)
context.setPositions(positions)
context.setVelocitiesToTemperature(temperature)

In [15]:
# Run eq forward (0 -> 1)
positions = []
energies = []
for step in range(int(nsteps_eq/10)):
#     print(step)
    integrator.step(10)


In [15]:
# Run neq forward (0 -> 1)
positions = []
for fwd_step in range(nsteps_neq):
#     if fwd_step <= 24990 and fwd_step % 1000 == 0: 
#         print(f"Forward neq: {fwd_step} completed")
#     if fwd_step > 24990:
#         pos = context.getState(getPositions=True).getPositions()
#         positions.append(list(pos.value_in_unit_system(unit.md_unit_system)))
#         components = compute_potential_components(context, beta=beta)
#         print(components)
# #         old = context.getParameter('lambda_0_sterics_old')
# #         new = context.getParameter('lambda_0_sterics_new')
# #         context.setParameter('lambda_0_sterics_old', abs(old))
# #         context.setParameter('lambda_0_sterics_new', abs(new))
#         print(context.getParameter('lambda_0_sterics_old'))
#         print(context.getParameter('lambda_0_sterics_new'))
    integrator.step(1)
    

In [16]:
# Run eq reverse (1 -> 0)
for step in range(nsteps_eq):
    integrator.step(1)



In [17]:
components = compute_potential_components(context, beta=beta)
print(components)


conducting subsequent work with the following platform: CUDA
[('MonteCarloBarostat', 0.0), ('CustomBondForce', 8.250559497189425), ('CustomAngleForce', 27.199583981810612), ('CustomTorsionForce', 22.470171094783556), ('CustomNonbondedForce', -15497.062482301315), ('CustomNonbondedForce', 1725.2468565236525), ('CustomBondForce', 70.8517274813265), ('CustomBondForce', 2.439606886753542)]


In [18]:
# Run neq reverse (1 -> 0)
for step in range(nsteps_neq):
    integrator.step(1)



KeyboardInterrupt: 

In [13]:
components = compute_potential_components(context, beta=beta)
print(components)

conducting subsequent work with the following platform: CUDA
[('MonteCarloBarostat', 0.0), ('CustomBondForce', 6.324543764574446), ('CustomAngleForce', 17.698552560004057), ('CustomTorsionForce', 16.68422532262658), ('CustomNonbondedForce', -15314.23872453846), ('CustomNonbondedForce', 1633.0162376459205), ('CustomBondForce', 82.30175829893093), ('CustomBondForce', 6.513541773900864)]
